In [15]:
%matplotlib widget
from bmcs_shell.folding.wb_model import WBModel
from bmcs_shell.folding.vmats2D_elastic import MATS2DElastic
import bmcs_utils.api as bu

In [18]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
n_phi_plus=4
n_x_plus=2
wbm = WBModel(F=-20000, h=10, tmodel=tmodel)
data = dict(alpha=1.24, a=500, a_high=3000, b=900, b_high=3000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus, show_nodes=False)
wbm.wb_mesh.trait_set(**data);

In [19]:
wbm.interact()

In [4]:
xmodel = wbm.fe_domain[0].xmodel
xmodel.mesh.I_CDij

array([[[[23, 22],
         [27, 26],
         [25, 24]],

        [[51, 50],
         [55, 54],
         [53, 52]]],


       [[[30, 29],
         [34, 33],
         [32, 31]],

        [[58, 57],
         [62, 61],
         [60, 59]]],


       [[[37, 36],
         [41, 40],
         [39, 38]],

        [[65, 64],
         [69, 68],
         [67, 66]]],


       [[[44, 43],
         [48, 47],
         [46, 45]],

        [[72, 71],
         [76, 75],
         [74, 73]]]])

In [5]:
_, idx_remap = xmodel.mesh.unique_node_map

In [6]:
idx_remap[xmodel.mesh.I_CDij]

array([[[[19, 18],
         [21,  2],
         [20,  6]],

        [[34, 33],
         [ 1, 36],
         [ 5, 35]]],


       [[[20,  6],
         [24,  4],
         [23, 11]],

        [[ 5, 35],
         [ 3, 39],
         [10, 38]]],


       [[[23, 11],
         [27,  9],
         [26, 16]],

        [[10, 38],
         [ 8, 42],
         [15, 41]]],


       [[[26, 16],
         [31, 14],
         [30, 29]],

        [[15, 41],
         [13, 46],
         [45, 44]]]])

In [7]:
idx_remap[xmodel.mesh.I_CDij[(0, -1),:,(0, -1),:].flatten()]

array([19, 18, 34, 33, 30, 29, 45, 44])

In [8]:
xmodel

In [9]:
wbm.interact()

In [10]:
import numpy as np
alpha_range = np.linspace(0.5, 1.3, 10)
a_range = np.linspace(300,500,10)

In [11]:
tmodel = MATS2DElastic(E=28000, nu=0.2)
import numpy as np
U_grid = []
sig_grid = []
for alpha in alpha_range:
    U_list = []
    sig_list = []
    for a in a_range:
        wbm = WBModel(F=-20000, h=10, tmodel=tmodel)
        data = dict(alpha=alpha, a=a, a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
                    n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
        wbm.wb_mesh.trait_set(**data)
        max_U = wbm.get_max_vals()
        
        U1 = wbm.hist.U_t[-1]
        eps1 = wbm.fe_domain[0].xmodel.map_U_to_field(U1)
        sig1, _ = wbm.fe_domain[0].tmodel.get_corr_pred(eps1,1)
        sig_x, sig_y, tau = sig1.T
        sig_ab = np.einsum('ab...->...ab', np.array([[sig_x, tau], [tau, sig_y]], dtype=np.float_))
        sig_val, sig_dir = np.linalg.eig(sig_ab)
        sig_list.append([np.min(sig_val), np.max(sig_val)])

        print(alpha, a, max_U)
        U_list.append(max_U)
    U_grid.append(U_list)
    sig_grid.append(sig_list)

0.5 300.0 7.583016453714706
0.5 322.22222222222223 7.800681817622978
0.5 344.44444444444446 8.056627589261682
0.5 366.66666666666663 8.358826600370564
0.5 388.8888888888889 8.717377303941058
0.5 411.1111111111111 9.145846720155735
0.5 433.3333333333333 9.662456725635641
0.5 455.55555555555554 10.29196375151712
0.5 477.77777777777777 11.069346743098485
0.5 500.0 12.045235195068496
0.5888888888888889 300.0 5.753776551498765
0.5888888888888889 322.22222222222223 5.924488318448107
0.5888888888888889 344.44444444444446 6.127736629435401
0.5888888888888889 366.66666666666663 6.370554291869256
0.5888888888888889 388.8888888888889 6.662075735007781
0.5888888888888889 411.1111111111111 7.014587303366085
0.5888888888888889 433.3333333333333 7.445033642488308
0.5888888888888889 455.55555555555554 7.976847791936937
0.5888888888888889 477.77777777777777 8.644095841773822
0.5888888888888889 500.0 9.497125393085568
0.6777777777777778 300.0 4.611177090837974
0.6777777777777778 322.22222222222223 4.755

In [10]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(alpha_range, a_range)
Z = np.array(U_grid)
surf = ax.plot_surface(X, Y, Z, 
                       color='green',
                       #linewidth=0, antialiased=False)
                    alpha=.0, linewidth=5,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
Z = np.array(sig_grid)[:,:,1]
surf = ax.plot_surface(X, Y, Z,color='red', 
                       #cmap=cm.copper,
                    alpha=.3, linewidth=3,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
Z = -np.array(sig_grid)[:,:,0]
surf = ax.plot_surface(X, Y, Z,color='blue', 
                       #cmap=cm.copper,
                    alpha=.6, linewidth=3,
                       #cmap=cm.gist_earth,antialiased=False, 
                shade=False,rstride=1, cstride=1)
ax.set_xlabel(r'$\alpha~[\mathrm{rad}]$')
ax.set_ylabel(r'$a~[\mathrm{mm}]$')
ax.set_zlabel(r'max compression [MPa]')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 0, 'max compression [MPa]')

In [67]:
i_alpha, i_a = 0, -1
wbm = WBModel(F=-1000, tmodel=tmodel)
print('alpha', alpha_range[i_alpha],'a', a_range[i_a] )
data = dict(alpha=alpha_range[i_alpha], 
            a=a_range[i_a], a_high=2000, b=1250, b_high=2000, c=400, c_high=2000,
            n_phi_plus=n_phi_plus, n_x_plus=n_x_plus)
wbm.wb_mesh.trait_set(**data)
wbm.wb_mesh.interact()

alpha 0.5 a 500.0


Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0, 0…

In [ ]:
15000 / 39000 